## Modules

In [ ]:
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F

from datetime import datetime

from google.oauth2 import service_account
from google.cloud import bigquery

## Hyper-parameters

In [ ]:
# model setting
max_len = 10
hidden_units = 50
num_heads = 1
num_layers = 2
dropout_rate=0.5
num_workers = 1
device = 'cuda' # gpu 환경 확인 필요

# training setting
lr = 0.001
batch_size = 128
num_epochs = 50

## Data preprocessing

In [3]:
SERVICE_ACCOUNT_FILE = "../../Data/config/level3-416207-893f91c9529e.json"  # 키 json 파일

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)

# 빅쿼리 클라이언트 객체 생성
project_id = "level3-416207"
client = bigquery.Client(credentials=credentials, project=project_id)

# 쿼리 실행
# 빅쿼리 디렉토리는 <프로젝트ID>.<데이터셋ID>.<테이블ID> 순으로 저장되어있음 ex) level3-416207.l3_30.l3_30
QUERY = (
    '''
    SELECT *
    FROM `level3-416207.log_129.ip_time_url_sorted_by_ip_time`
    ''')


# API request
df = client.query(QUERY).to_dataframe()

In [4]:
df = df[['hashed_ip', 'local_time', 'request_url_endpoint']]
df['local_time'] = pd.to_datetime(df['local_time'], format='%d/%b/%Y:%H:%M:%S')
df['timestamp']=pd.to_datetime(df['local_time']).astype(int)//10**9
df = df[['hashed_ip', 'request_url_endpoint', 'timestamp']]

In [5]:
spec = df[df['request_url_endpoint'].str.startswith('/spec')]
comment = df[df['request_url_endpoint'].str.startswith('/comments')]
product = df[df['request_url_endpoint'].str.startswith('/products')]

df = pd.concat([product])
#df = pd.concat([spec, comment, product])

df = df.sort_values(['hashed_ip', 'timestamp'])

In [6]:
df['product_id'] = df['request_url_endpoint'].str.split('/').str[2]
df = df[df['product_id'].apply(lambda x: len(str(x)) == len('db696dbc255f0443bb7f782ac0ec24d45003f792cb9dbb7c810f7dd8216a18b2'))]

In [7]:
df['user']=df['hashed_ip']
df['item']=df['product_id']
df['time']=df['timestamp']

del df['hashed_ip'], df['request_url_endpoint'], df['timestamp'], df['product_id']

In [8]:
df

,user,item,time
6277936,00012dd418cdd7e02ff8b1a54f0a8d69,63147c6e78fcf3360b76cb06745efa373dd7e08cbf92b7...,1698341
5067189,00012dd418cdd7e02ff8b1a54f0a8d69,f774892f67793bf7218b23c5b72ba7ee481f51d3dff647...,1698781
5117086,00012dd418cdd7e02ff8b1a54f0a8d69,f6abf3c612d5c21d0ab89f499c1a5629a0ae0d1627426c...,1698781
6098168,00012dd418cdd7e02ff8b1a54f0a8d69,92bc40d09c604fb8dbb2d283959e0c4b2ba98b984eac1b...,1698782
4848645,00012dd418cdd7e02ff8b1a54f0a8d69,6fdae04380f97c32942d34657e0ca2f5853c7dd02557ce...,1702406
...,...,...,...
9403146,fffd7e8a1ab612e28dc13fe8f544379e,0dd8b052665d7eba05a212ed2945c10fe360239420285e...,1704570
9850286,fffee8de197f26f89f965c2ea16c3ccd,4e625ee7f03e937a876d5462ba2c06fed229000c303fb4...,1699133
6994109,ffff620421fb3729200f091926ee2d3d,5fa517a74273fb46a3dbd5224cb7e49c8a6011c767f703...,1696896
8098253,ffff620421fb3729200f091926ee2d3d,5fa517a74273fb46a3dbd5224cb7e49c8a6011c767f703...,1696896


In [9]:
# user별 interaction 개수를 계산
user_interaction_counts = df['user'].value_counts()

# interaction이 3개 이상인 user의 목록을 가져옴
selected_users = user_interaction_counts[user_interaction_counts >= 3].index

# interaction이 3개 이상인 user에 대한 데이터만 남김
df = df[df['user'].isin(selected_users)]

In [10]:
item_ids = df['item'].unique()
user_ids = df['user'].unique()
num_item, num_user = len(item_ids), len(user_ids)
num_batch = num_user // batch_size

# user, item indexing
item2idx = pd.Series(data=np.arange(len(item_ids))+1, index=item_ids) # item re-indexing (1~num_item)
user2idx = pd.Series(data=np.arange(len(user_ids)), index=user_ids) # user re-indexing (0~num_user-1)

# dataframe indexing
df = pd.merge(df, pd.DataFrame({'item': item_ids, 'item_idx': item2idx[item_ids].values}), on='item', how='inner')
df = pd.merge(df, pd.DataFrame({'user': user_ids, 'user_idx': user2idx[user_ids].values}), on='user', how='inner')
df.sort_values(['user_idx', 'time'], inplace=True)
del df['item'], df['user']

# train set, valid set 생성
users = defaultdict(list) # defaultdict은 dictionary의 key가 없을때 default 값을 value로 반환
user_train = {}
user_valid = {}
for u, i, t in zip(df['user_idx'], df['item_idx'], df['time']):
    users[u].append(i)

for user in users:
    user_train[user] = users[user][:-1]
    user_valid[user] = [users[user][-1]]

print(f'num users: {num_user}, num items: {num_item}')

num users: 33075, num items: 35088


In [11]:
# for training, data sampling
def random_neg(l, r, s):
    # log에 존재하는 아이템과 겹치지 않도록 sampling
    t = np.random.randint(l, r)
    while t in s:
        t = np.random.randint(l, r)
    return t

def sample_batch(user_train, num_user, num_item, batch_size, max_len):
    def sample():

        user = np.random.randint(num_user)  # user를 임의로 선택

        # 미리 max_len에 해당하는 array 생성, zero padding
        seq = np.zeros([max_len], dtype=np.int32)
        pos = np.zeros([max_len], dtype=np.int32)
        neg = np.zeros([max_len], dtype=np.int32)
        nxt = user_train[user][-1]
        idx = max_len - 1

        # negative sample은 train sequence에 없는 item 사용
        train_item = set(user_train[user])
        for i in reversed(user_train[user][:-1]):
            # 미리 정의된 sequence를 역순으로 채움, ex: seq = [0,0,0,1,2,3] (0은 pad)
            seq[idx] = i
            pos[idx] = nxt
            if nxt != 0:
                neg[idx] = random_neg(1, num_item + 1, train_item)
            nxt = i
            idx -= 1
            if idx == -1: break
        return (user, seq, pos, neg)
    user, seq, pos, neg = zip(*[sample() for _ in range(batch_size)])
    user, seq, pos, neg = np.array(user), np.array(seq), np.array(pos), np.array(neg)
    return user, seq, pos, neg

## Model

### Self attention

In [12]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super(ScaledDotProductAttention, self).__init__()
        self.hidden_units = hidden_units
        self.dropout = nn.Dropout(dropout_rate) # TODO1: dropout rate를 hyper parameter로 사용하여 dropout layer를 구현하세요.

    def forward(self, Q, K, V, mask):
        attn_score = torch.matmul(Q, K.transpose(2, 3)) / math.sqrt(self.hidden_units)
        attn_score = attn_score.masked_fill(mask == 0, -1e9)  # 유사도가 0인 지점은 -infinity로 보내 softmax 결과가 0이 되도록 함
        attn_dist = self.dropout(F.softmax(attn_score, dim=-1))  # attention distribution
        output = torch.matmul(attn_dist, V)  # dim of output : batchSize x num_head x seqLen x hidden_units
        return output, attn_dist

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, hidden_units, dropout_rate):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads # head의 수
        self.hidden_units = hidden_units

        # query, key, value, output 생성을 위해 Linear 모델 생성
        self.W_Q = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_K = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_V = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_O = nn.Linear(hidden_units, hidden_units, bias=False)

        self.attention = ScaledDotProductAttention(hidden_units, dropout_rate) # TODO2: ScaledDotProductAttention class를 사용하여 attention layer를 구현하세요.
        self.dropout = nn.Dropout(dropout_rate) # dropout rate
        self.layerNorm = nn.LayerNorm(hidden_units, 1e-6) # layer normalization

    def forward(self, enc, mask):
        residual = enc # residual connection을 위해 residual 부분을 저장
        batch_size, seqlen = enc.size(0), enc.size(1)

        # Query, Key, Value를 (num_head)개의 Head로 나누어 각기 다른 Linear projection을 통과시킴
        Q = self.W_Q(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)
        K = self.W_K(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)
        V = self.W_V(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)

        # Head별로 각기 다른 attention이 가능하도록 Transpose 후 각각 attention에 통과시킴
        Q, K, V = Q.transpose(1, 2), K.transpose(1, 2), V.transpose(1, 2)
        output, attn_dist = self.attention(Q, K, V, mask)

        # 다시 Transpose한 후 모든 head들의 attention 결과를 합칩니다.
        output = output.transpose(1, 2).contiguous()
        output = output.view(batch_size, seqlen, -1)

        # Linear Projection, Dropout, Residual sum, and Layer Normalization
        output = self.layerNorm(self.dropout(self.W_O(output)) + residual)
        return output, attn_dist

### Position-wise Feed Forward Network


In [13]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super(PositionwiseFeedForward, self).__init__()

        self.W_1 = nn.Linear(hidden_units, hidden_units)
        self.W_2 = nn.Linear(hidden_units, hidden_units)
        self.dropout = nn.Dropout(dropout_rate)
        self.layerNorm = nn.LayerNorm(hidden_units, 1e-6) # layer normalization

    def forward(self, x):
        residual = x
        output = self.W_2(F.relu(self.dropout(self.W_1(x))))
        output = self.layerNorm(self.dropout(output) + residual)
        return output

In [14]:
class SASRecBlock(nn.Module):
    def __init__(self, num_heads, hidden_units, dropout_rate):
        super(SASRecBlock, self).__init__()
        self.attention = MultiHeadAttention(num_heads, hidden_units, dropout_rate)
        self.pointwise_feedforward = PositionwiseFeedForward(hidden_units, dropout_rate)

    def forward(self, input_enc, mask):
        output_enc, attn_dist = self.attention(input_enc, mask)
        output_enc = self.pointwise_feedforward(output_enc)
        return output_enc, attn_dist

### SASRec

In [15]:
class SASRec(nn.Module):
    def __init__(self, num_user, num_item, hidden_units, num_heads, num_layers, maxlen, dropout_rate, device):
        super(SASRec, self).__init__()

        self.num_user = num_user
        self.num_item = num_item
        self.hidden_units = hidden_units
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.device = device

        self.item_emb = nn.Embedding(num_item + 1, hidden_units, padding_idx=0) # TODO3: item embedding을 생성하세요. (padding index 고려 필요)
        self.pos_emb = nn.Embedding(maxlen, hidden_units) # learnable positional encoding
        self.dropout = nn.Dropout(dropout_rate)
        self.emb_layernorm = nn.LayerNorm(hidden_units, eps=1e-6)

        self.blocks = nn.ModuleList([SASRecBlock(num_heads, hidden_units, dropout_rate) for _ in range(num_layers)])

    def feats(self, log_seqs):
        seqs = self.item_emb(torch.LongTensor(log_seqs).to(self.device))
        positions = np.tile(np.array(range(log_seqs.shape[1])), [log_seqs.shape[0], 1])
        seqs += self.pos_emb(torch.LongTensor(positions).to(self.device))
        seqs = self.emb_layernorm(self.dropout(seqs))

        # masking
        mask_pad = torch.BoolTensor(log_seqs > 0).unsqueeze(1).unsqueeze(1) # TODO4: log_seqs=0인 경우 masking이 필요합니다. 해당 조건을 만족하는 mask를 구현하세요.
        # 참고 unsqueeze operation을 통해 mask_pad와 mask_time의 차원을 맞춰주는 과정이 필요합니다.
        mask_time = (1 - torch.triu(torch.ones((1, 1, seqs.size(1), seqs.size(1))), diagonal=1)).bool() # sequence의 순서를 고려
        mask = (mask_pad & mask_time).to(self.device)
        for block in self.blocks:
            seqs, attn_dist = block(seqs, mask)
        return seqs

    def forward(self, log_seqs, pos_seqs, neg_seqs):
        # 학습에 사용
        feats = self.feats(log_seqs) # TODO5: Transformer를 사용해서 token 별 연산을 수행하세요.
        pos_embs = self.item_emb(torch.LongTensor(pos_seqs).to(self.device))
        neg_embs = self.item_emb(torch.LongTensor(neg_seqs).to(self.device))

        pos_logits = (feats * pos_embs).sum(dim=-1)
        neg_logits = (feats * neg_embs).sum(dim=-1)
        return pos_logits, neg_logits

    def predict(self, log_seqs, item_indices):
        # evaluation에 사용
        final_feats = self.feats(log_seqs)[:, -1, :]
        item_embs = self.item_emb(torch.LongTensor(item_indices).to(self.device))
        logits = item_embs.matmul(final_feats.unsqueeze(-1)).squeeze(-1)
        return logits

## Training

In [16]:
# setting
model = SASRec(num_user, num_item, hidden_units, num_heads, num_layers, max_len, dropout_rate, device)
model.to(device)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [17]:
# training
for epoch in range(1, num_epochs + 1):
    tbar = tqdm(range(num_batch))
    for step in tbar: # num_batch만큼 sampling
        user, seq, pos, neg = sample_batch(user_train, num_user, num_item, batch_size, max_len)
        pos_logits, neg_logits = model(seq, pos, neg)
        pos_labels, neg_labels = torch.ones(pos_logits.shape, device=device), torch.zeros(neg_logits.shape, device=device)

        optimizer.zero_grad()
        indices = np.where(pos != 0)
        loss = criterion(pos_logits[indices], pos_labels[indices])
        loss += criterion(neg_logits[indices], neg_labels[indices])
        loss.backward()
        optimizer.step()

        tbar.set_description(f'Epoch: {epoch:3d}| Step: {step:3d}| Train loss: {loss:.5f}')

Epoch:   1| Step: 257| Train loss: 3.02935: 100%|██████████| 258/258 [00:01<00:00, 130.88it/s]
Epoch:   2| Step: 257| Train loss: 1.61470: 100%|██████████| 258/258 [00:01<00:00, 151.96it/s]
Epoch:   3| Step: 257| Train loss: 1.31541: 100%|██████████| 258/258 [00:01<00:00, 148.32it/s]
Epoch:   4| Step: 257| Train loss: 1.31490: 100%|██████████| 258/258 [00:01<00:00, 148.44it/s]
Epoch:   5| Step: 257| Train loss: 1.18575: 100%|██████████| 258/258 [00:01<00:00, 148.66it/s]
Epoch:   6| Step: 257| Train loss: 1.11722: 100%|██████████| 258/258 [00:01<00:00, 149.25it/s]
Epoch:   7| Step: 257| Train loss: 1.02641: 100%|██████████| 258/258 [00:01<00:00, 147.83it/s]
Epoch:   8| Step: 257| Train loss: 0.98343: 100%|██████████| 258/258 [00:01<00:00, 149.94it/s]
Epoch:   9| Step: 257| Train loss: 0.85962: 100%|██████████| 258/258 [00:01<00:00, 149.59it/s]
Epoch:  10| Step: 257| Train loss: 0.89668: 100%|██████████| 258/258 [00:01<00:00, 149.11it/s]
Epoch:  11| Step: 257| Train loss: 0.77085: 100%|█

## Evaluate

In [19]:
model.eval()
NDCG5 = 0.0 # NDCG@5
HIT5 = 0.0 # HIT@5
NDCG10 = 0.0 # NDCG@10
HIT10 = 0.0 # HIT@10
num_item_sample = num_item
num_user_sample = num_user
users = np.random.randint(0, num_user, num_user_sample) # 1000개만 sampling 하여 evaluation
for u in users:
    seq = user_train[u][-max_len:]
    rated = set(user_train[u] + user_valid[u])
    item_idx = user_valid[u] + [random_neg(1, num_item + 1, rated) for _ in range(num_item_sample)]
    with torch.no_grad():
        predictions = -model.predict(np.array([seq]), np.array(item_idx))
        predictions = predictions[0]
        rank = predictions.argsort().argsort()[0].item()
    if rank < 5: # 만약 예측 성공시
        NDCG5 += 1 / np.log2(rank + 2)
        HIT5 += 1
    if rank < 10: # 만약 예측 성공시
        NDCG10 += 1 / np.log2(rank + 2)
        HIT10 += 1
print(f'NDCG@5: {NDCG5/num_user_sample}| HIT@5: {HIT5/num_user_sample}')
print(f'NDCG@10: {NDCG10/num_user_sample}| HIT@10: {HIT10/num_user_sample}')

NDCG@10: 0.19438740744031202| HIT@10: 0.287
